In [2]:
import numpy as np
import matplotlib.pyplot as pyplot
import glob
import cv2
import tensorflow as tf

In [3]:
print(tf.__version__)

2.3.0


In [4]:
#!pip install tensorflow-gpu==2.2.0

In [5]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [6]:
#Now we read the input images and assign labels according to the folder names

print(os.listdir("C:/Users/Kaustuv/BreastCancer"))
SIZE=256#Resizing all the images to 256 for VGG16 image feature extraction

['test', 'train', 'valid']


In [7]:
folders=glob.glob('C:/Users/Kaustuv/BreastCancer/train/*')
print(folders)

['C:/Users/Kaustuv/BreastCancer/train\\0_NoCancer', 'C:/Users/Kaustuv/BreastCancer/train\\1_Cancer']


In [8]:
#Now we store the training data and labels into respective lists
train_images=[]
train_labels=[]

In [9]:
for directory_path in glob.glob("C:/Users/Kaustuv/BreastCancer/train/*"):
  label=directory_path.split("//")[-1]
  print(label)
  for img_path in glob.glob(os.path.join(directory_path,"*.png")):
    print(img_path)
    img=cv2.imread(img_path,cv2.IMREAD_COLOR)
    img=cv2.resize(img,(SIZE,SIZE))
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    train_images.append(img)
    train_labels.append(label)

C:/Users/Kaustuv/BreastCancer/train\0_NoCancer
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1001_y1301_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1101_y801_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1151_y1251_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1201_y2001_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1251_y1751_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1251_y51_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1301_y301_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1301_y351_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1301_y751_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1301_y901_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1351_y1951_class0.png
C:/Users/Kaustuv/BreastCancer/train\0_NoCancer\8863_idx5_x1401_y1351_class0.png


In [11]:
#Converting to numpy arrays
train_images=np.asarray(train_images)
train_labels=np.asarray(train_labels)

In [12]:
train_labels

array(['C:/Users/Kaustuv/BreastCancer/train\\0_NoCancer',
       'C:/Users/Kaustuv/BreastCancer/train\\0_NoCancer',
       'C:/Users/Kaustuv/BreastCancer/train\\0_NoCancer', ...,
       'C:/Users/Kaustuv/BreastCancer/train\\1_Cancer',
       'C:/Users/Kaustuv/BreastCancer/train\\1_Cancer',
       'C:/Users/Kaustuv/BreastCancer/train\\1_Cancer'], dtype='<U46')

In [13]:
#Capturing Validation dataset into respective lists

val_images=[]
val_labels=[]

for directory_path in glob.glob("C:/Users/Kaustuv/BreastCancer/valid/*"):
  label=directory_path.split("//")[-1]
  print(label)
  for img_path in glob.glob(os.path.join(directory_path,"*.png")):
    print(img_path)
    img=cv2.imread(img_path,cv2.IMREAD_COLOR)
    img=cv2.resize(img,(SIZE,SIZE))
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    val_images.append(img)
    val_labels.append(label)

C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1001_y1651_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1101_y1751_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1251_y2051_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1301_y451_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1351_y751_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1401_y1901_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1451_y1901_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x1801_y751_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x2101_y1751_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x2301_y1751_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x2501_y1451_class0.png
C:/Users/Kaustuv/BreastCancer/valid\0_NoCancer\8863_idx5_x2601_y1701_class0.

In [14]:
#Converting to numpy array
val_images=np.asarray(val_images)
val_labels=np.asarray(val_labels)

In [15]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
le.fit(val_labels)
val_labels_encoded=le.transform(val_labels)
le.fit(train_labels)
train_labels_encoded=le.transform(train_labels)

In [16]:
print(train_labels_encoded)

[0 0 0 ... 1 1 1]


In [17]:
#Split the data into training and testing 
x_train,y_train,x_val,y_val=train_images,train_labels_encoded,val_images,val_labels_encoded

In [18]:
#Normalise the pixel values between 0 to 1 
x_train,x_val=x_train/255.0,x_val/255.0

In [19]:
print(x_train)

[[[[0.78431373 0.4        0.54901961]
   [0.78431373 0.4        0.54901961]
   [0.78431373 0.4        0.54901961]
   ...
   [0.77647059 0.45490196 0.59607843]
   [0.77647059 0.45490196 0.59607843]
   [0.77647059 0.45490196 0.59607843]]

  [[0.78431373 0.4        0.54901961]
   [0.78431373 0.4        0.54901961]
   [0.78431373 0.4        0.54901961]
   ...
   [0.77647059 0.45490196 0.59607843]
   [0.77647059 0.45490196 0.59607843]
   [0.77647059 0.45490196 0.59607843]]

  [[0.78431373 0.4        0.54901961]
   [0.78431373 0.4        0.54901961]
   [0.78431373 0.4        0.54901961]
   ...
   [0.77647059 0.45490196 0.59607843]
   [0.77647059 0.45490196 0.59607843]
   [0.77647059 0.45490196 0.59607843]]

  ...

  [[0.79607843 0.45882353 0.60784314]
   [0.79607843 0.45882353 0.60784314]
   [0.79607843 0.45882353 0.60784314]
   ...
   [0.72156863 0.40784314 0.56078431]
   [0.72156863 0.40784314 0.56078431]
   [0.72156863 0.40784314 0.56078431]]

  [[0.79607843 0.45882353 0.60784314]
   [0.7

In [20]:
#Load model into VGG16 without classifier for feature extraction
VGG_model=VGG16(weights='imagenet',include_top=False,input_shape=(SIZE,SIZE,3))

In [21]:
for layer in VGG_model.layers:
  layer.trainable=False

In [22]:
VGG_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [23]:
#Feature Extraction
feature_extractor=VGG_model.predict(x_train)
features=feature_extractor.reshape(feature_extractor.shape[0],-1)

In [24]:
x_training=features

In [37]:
from sklearn import svm
model=svm.SVC(kernel='rbf',gamma=0.1)

In [38]:
model.fit(x_training,train_labels_encoded)

SVC(gamma=0.1)

In [39]:
#Getting Validation Features
x_val_feature=VGG_model.predict(x_val)
x_val_features=x_val_feature.reshape(x_val_feature.shape[0],-1)

In [40]:
prediction=model.predict(x_val_features)

In [41]:
prediction

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [42]:
from sklearn import metrics
print("Accuracy = ", metrics.accuracy_score(prediction,val_labels_encoded))

Accuracy =  0.5083333333333333


In [33]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(x_training,train_labels_encoded)

In [34]:
accuracy = grid_search.best_score_
grid_search.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}